In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [ ]:
#DATA sets
#shapefiles
unimodal_shapefile='rheas/gadm36_TZA_LongRain.shp'
bimodal_shapefile='rheas/gadm36_TZA_2_Bimodal.shp'
country_shapefile='rheas/gadm36_TZA_0.shp'
unimodal_csv='rheas/tanzania_tamsat_25km_districts_unimod_dssatTable_2021_2023_100kg.csv'
bimodal_csv='rheas/tanzania_tamsat_25km_districts_bimod_dssatTable_2000_2023_100kg.csv'

In [70]:
def format_df(filename):
    df = pd.read_csv(filename)
    df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
    df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
    df['date'] = df['harvest'].dt.strftime("%Y")
    df = df.rename(columns={'cname': 'Region'})
    #df = df.rename(columns={df.columns[1]: "Region"})
    #df['Region'] = df['Region'].str.upper()
    return df

In [71]:
unimodal_data = format_df(unimodal_csv)

In [72]:
# Unimodal season:  October--August (LR)
lrStart = [9, 10, 11, 12, 1,2, 3, 4]
lrEnd = [2, 3, 4, 5, 6, 7, 8]
def LR_metrics(lr, lrStart, lrEnd, season):
    lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
    lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return lr

rh_lr = LR_metrics(unimodal_data, lrStart, lrEnd, "LR")
print(rh_lr.head())

     Region  date Season         gwad
0      Bahi  2022     LR  3783.650000
1      Bahi  2023     LR  3123.037500
2   Buhigwe  2022     LR  3725.214286
3   Buhigwe  2023     LR  2691.573529
4  Chamwino  2022     LR  3743.050000


In [73]:
# Bimodal season:  October--August (OND)
bimodal_data = format_df(bimodal_csv)
ondStart = [9, 10, 11]
ondEnd = [1, 2, 3]
def OND_metrics(ond, ondStart, ondEnd, season):
    ond.loc[(pd.to_datetime(ond['planting']).dt.month.isin(ondStart)) & (pd.to_datetime(ond['harvest']).dt.month.isin(ondEnd)), 'Season'] = season
    ond = ond.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return ond

rh_ond = OND_metrics(bimodal_data, ondStart, ondEnd, "OND")
print(rh_ond.head())

   Region  date Season      gwad
0  Arusha  2000    OND  3205.300
1  Arusha  2001    OND  3062.425
2  Arusha  2002    OND  3394.625
3  Arusha  2003    OND  3228.975
4  Arusha  2004    OND  3246.025


In [74]:
# Read shapefiles and merge data
unimodal_shapefile = gpd.read_file("rheas/gadm36_TZA_2_LongRain.shp")
country_shapefile = gpd.read_file("rheas/gadm36_TZA_0.shp")
unimodal_shapefile.rename(columns={'NAME_2':'Region'}, inplace=True)
unimodal_shapefile = unimodal_shapefile[['Region']]
rh_lr = rh_lr[['Region','Season','gwad']]
unimodal_merged_data = rh_lr.merge(unimodal_shapefile, on="Region", how='left')
unimodal_merged_data.head()

,Region,Season,gwad
0,Bahi,LR,3783.650000
1,Bahi,LR,3123.037500
2,Buhigwe,LR,3725.214286
3,Buhigwe,LR,2691.573529
4,Chamwino,LR,3743.050000


In [76]:
geo_source = GeoJSONDataSource(geojson=unimodal_merged_data.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [77]:
color_mapper = LinearColorMapper(palette=Viridis256, low=min(unimodal_merged_data['gwad']), high=max(unimodal_merged_data['gwad']))

In [78]:
p1 = figure(title='Yield Data by District', plot_width=650, plot_height=600)

p1.multi_line('xs','ys', source=country_geo_source, line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source,
          fill_color={'field': 'gwad', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@NAME_2'), ('gwad', '@gwad')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [79]:
show(p1)

In [ ]:
color_mapper1 = LinearColorMapper(palette=Viridis256, low=min(unimodal_merged_data['gwad']), high=max(unimodal_merged_data['gwad']))

p2 = figure(title='Yield Data by District',
           plot_width=650, plot_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source,
          fill_color={'field': 'gwad', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@NAME_2'), ('gwad', '@gwad')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [ ]:
#from bokeh.layouts import column
#layout = column(p1, p2)
#show(layout)

In [ ]:
#from bokeh.io import output_file, show
#output_file(plot.html)
show(layout)

In [ ]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename=yield_map.html, title=Yield Data Map)